# Análise dos acidentes de trânsito na Nigéria entre 2020 e 2024
Utilizando PySpark

### Importanto e configurando o ambiente para uso da biblioteca PySpark

In [26]:
# Importando as bibliotecas que usaremos para ter a localização do pyspark
import findspark
import os

# Configura o caminho do Spark
os.environ["SPARK_HOME"] = '/Users/marcusferreira/Documents/Vscode/PySpark/spark-3.4.0-bin-hadoop3'

# Inicializa o findspark, para ele localizar onde está armazenado
findspark.init()

# Biblioteca do Spark para utilização de DataFrame e linguagem SQL
from pyspark.sql import SparkSession

# Importando as funções SQL do PySpark para utilizarmos ao longo do código
from pyspark.sql.functions import col, isnan, when, count
from pyspark.sql.types import IntegerType

# Inicializa uma SparkSession com um nome de aplicação descritivo
spark = SparkSession.builder \
    .appName("AnaliseAcidentesTransito") \
    .master('local[*]') \
    .getOrCreate()

# Obtém o SparkContext a partir da SparkSession
sc = spark.sparkContext

### Obtendo o arquivo e fazendo algumas análises iniciais

In [27]:
# Obtendo o arquivo CSV e criando um DataFrame a partir dele
path = '/Users/marcusferreira/Documents/Vscode/Traffic_crashes/NigerianTrafficCrashes.csv'
df = spark.read.option('delimiter', ',').option("header", True).csv(path)

# Visualizando o DataFrame
df.show()

# Verificação dos tipos de campos
df.printSchema()

# Verificando os casos nulos
null_counts = df.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df.columns])
null_counts.show()

+-------+-----------+-------------+-----------+----------+-----------------------+---+---+---+---+-------------+
|Quarter|      State|Total_Crashes|Num_Injured|Num_Killed|Total_Vehicles_Involved|SPV|DAD|PWR|FTQ|Other_Factors|
+-------+-----------+-------------+-----------+----------+-----------------------+---+---+---+---+-------------+
|Q4 2020|       Abia|           30|        146|        31|                     37| 19|  0|  0|  0|           18|
|Q4 2020|    Adamawa|           77|        234|        36|                     94| 57|  0|  0|  0|           37|
|Q4 2020|  Akwa Ibom|           22|         28|         7|                     24| 15|  0|  0|  1|            8|
|Q4 2020|    Anambra|           72|        152|        20|                     83| 43|  1|  0|  0|           39|
|Q4 2020|     Bauchi|          154|        685|        90|                    140| 74|  0|  0|  0|           66|
|Q4 2020|    Bayelsa|            1|          3|         0|                      1|  1|  0|  0|  

### Aplicando tratamento de dados

In [28]:
# Alterando os tipos das colunas para o formato correto
df = df.withColumn('Total_Crashes', df['Total_Crashes'].cast(IntegerType()))
df = df.withColumn('Num_Injured', df['Num_Injured'].cast(IntegerType()))
df = df.withColumn('Num_Killed', df['Num_Killed'].cast(IntegerType()))
df = df.withColumn('Total_Vehicles_Involved', df['Total_Vehicles_Involved'].cast(IntegerType()))
df = df.withColumn('SPV', df['SPV'].cast(IntegerType()))
df = df.withColumn('DAD', df['DAD'].cast(IntegerType()))
df = df.withColumn('PWR', df['PWR'].cast(IntegerType()))
df = df.withColumn('FTQ', df['FTQ'].cast(IntegerType()))
df = df.withColumn('Other_Factors', df['Other_Factors'].cast(IntegerType()))
df.printSchema()
df.show()

root
 |-- Quarter: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Total_Crashes: integer (nullable = true)
 |-- Num_Injured: integer (nullable = true)
 |-- Num_Killed: integer (nullable = true)
 |-- Total_Vehicles_Involved: integer (nullable = true)
 |-- SPV: integer (nullable = true)
 |-- DAD: integer (nullable = true)
 |-- PWR: integer (nullable = true)
 |-- FTQ: integer (nullable = true)
 |-- Other_Factors: integer (nullable = true)

+-------+-----------+-------------+-----------+----------+-----------------------+---+---+---+---+-------------+
|Quarter|      State|Total_Crashes|Num_Injured|Num_Killed|Total_Vehicles_Involved|SPV|DAD|PWR|FTQ|Other_Factors|
+-------+-----------+-------------+-----------+----------+-----------------------+---+---+---+---+-------------+
|Q4 2020|       Abia|           30|        146|        31|                     37| 19|  0|  0|  0|           18|
|Q4 2020|    Adamawa|           77|        234|        36|                     94| 57| 

In [29]:
# Parar a SparkSession
spark.stop()